In [2]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
#import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import evaluate
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,DataCollatorForTokenClassification,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer,TFRobertaForTokenClassification,TFAutoModelForTokenClassification,)

In [3]:
#pip install evaluate datasets

In [4]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2024-11-12 00:40:03--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  3.97MB/s    in 1.9s    

2024-11-12 00:40:06 (3.97 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]



In [5]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


In [6]:
#from datasets import load_dataset
dataset=load_dataset('text', data_files='/content/dataset/fra.txt')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 232736
    })
})

In [8]:
tmp=dataset['train']['text'][:2]

In [9]:
for i in tmp:
  print(i)

Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)


In [10]:
for i in tmp:
  print(i.split('\t')[:2])

['Go.', 'Va !']
['Go.', 'Marche.']


In [11]:
from transformers import (DataCollatorWithPadding,create_optimizer,AutoTokenizer,DataCollatorForSeq2Seq,
                          T5TokenizerFast,T5ForConditionalGeneration,TFAutoModelForSeq2SeqLM)

In [12]:
model_id='t5-small'
tokenizer=T5TokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [13]:

prefix = "translate English to French: "
def preprocess_function(examples):

  inputs = [prefix + example.split('\t')[0] for example in examples['text']]
  targets = [example.split('\t')[1] for example in examples['text']]

  model_inputs = tokenizer(inputs, text_target=targets,max_length=128, truncation=True)
  return model_inputs

In [14]:
tokenized_dataset=dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/232736 [00:00<?, ? examples/s]

In [15]:
max_len=128
batch_size=16

In [16]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 232736
    })
})

In [17]:
tokenized_dataset=tokenized_dataset.remove_columns(['text'])

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 232736
    })
})

In [19]:
model=TFAutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model,return_tensors='tf')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [20]:
train_dataset=tokenized_dataset['train'].to_tf_dataset(
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [21]:
train_data=train_dataset.take(int(0.9*len(train_dataset)))
val_data=train_dataset.skip(int(0.9*len(train_dataset)))

In [23]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
num_epochs = 3
num_train_steps = len(train_data) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)

In [25]:
history=model.fit(
  train_data,
  validation_data=val_data,
  epochs=1
)

13091/13091 [==============================] - 1700s 127ms/step - loss: 0.8046 - val_loss: 0.5966


In [29]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [31]:
def translate_text(sentence, model, tokenizer, max_length=128):
    # Prepare the input by adding the translation prompt, if necessary
    input_text = f"translate English to French: {sentence}"

    # Tokenize the input sentence
    inputs = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True, max_length=max_length)

    # Generate the translation
    output_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask, max_length=max_length, num_beams=4, early_stopping=True)

    # Decode the generated tokens
    translated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return translated_text


In [32]:
# Example usage
english_sentence = "Hello, how are you?"
translated_sentence = translate_text(english_sentence, model, tokenizer)
print("English:", english_sentence)
print("French Translation:", translated_sentence)


English: Hello, how are you?
French Translation: Bonjour, comment êtes-vous?
